In [2]:
# Instalar Selenium, BeautifulSoup, Pandas y Webdriver Manager
!pip install selenium beautifulsoup4 pandas webdriver-manager

# --- INSTALACIÓN DE GOOGLE CHROME EN COLAB ---
# 1. Descargar la clave GPG de Google Chrome
!wget -q -O - https://dl.google.com/linux/linux_signing_key.pub | sudo gpg --dearmor -o /usr/share/keyrings/google-chrome-archive-keyring.gpg

# 2. Añadir el repositorio de Google Chrome a las fuentes de apt
!echo "deb [arch=amd64 signed-by=/usr/share/keyrings/google-chrome-archive-keyring.gpg] http://dl.google.com/linux/chrome/deb/ stable main" | sudo tee /etc/apt/sources.list.d/google-chrome.list

# 3. Actualizar los listados de paquetes apt para incluir el nuevo repositorio
!sudo apt-get update

# 4. Instalar Google Chrome estable
!sudo apt-get install -y google-chrome-stable

# Opcional: Verificar la versión de Chrome instalada
!google-chrome --version

print("\n--- Instalación de Chrome y dependencias completada ---")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.4.0
    Uninstalling urllib3-2.4.0:
      Successfully uninstalled urllib3-2.4.0
deb [arch=amd64 signed-by=/usr/share/keyrings/google-chrome-archive-keyring.gpg] http://dl.google.com/linux/chrome/deb/ stable main
Get:1 http://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,212 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://ar

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
from webdriver_manager.chrome import ChromeDriverManager

def extraer_planes_movistar_colab():
    url = "https://www.movistar.com.pe/movil/postpago/planes-postpago"
    planes_data = []

    # --- CONFIGURACIÓN DE SELENIUM PARA COLAB ---
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--log-level=3')
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    driver = None

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        print(f"Navegador abierto en Colab y cargando la página: {url}")

        # --- ESPERAR A QUE EL CONTENIDO DINÁMICO CARGUE ---
        wait = WebDriverWait(driver, 30)
        try:
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'p-plan__slide__soles')))
            print("Elemento de precio detectado. Esperando que carguen más elementos...")
        except:
            print("No se encontró el elemento de precio inicialmente. Intentando scroll y reintento.")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'p-plan__slide__soles')))

        time.sleep(5)

        print("Página cargada y elementos principales detectados. Extrayendo HTML...")

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        plan_elements = soup.find_all('div', class_='p-plan__slide__shadow')

        if not plan_elements:
            print("ERROR: No se encontraron elementos con la clase 'p-plan__slide__shadow'.")
            print("Esto podría indicar que la clase ha cambiado nuevamente o el contenido no se cargó como se esperaba.")
            return []

        print(f"Se encontraron {len(plan_elements)} posibles contenedores de planes.")

        for i, plan_element in enumerate(plan_elements):
            nombre_plan = 'N/A'
            precio = 'N/A'
            gigas = 'N/A'
            apps_ilimitadas_text = 'N/A'
            otros_beneficios = {}
            llamadas_encontradas = None
            sms_encontrados = None

            # 1. Extraer Precio
            precio_tag = plan_element.find('span', class_='p-plan__slide__soles')
            if precio_tag:
                precio = precio_tag.get_text(strip=True)

            # 2. Extraer Nombre del Plan (Limpieza Mejorada)
            head_tag = plan_element.find('div', class_='p-plan__slide__head')
            if head_tag:
                name_tag = head_tag.find(['h3', 'h4', 'span', 'p'], class_=lambda x: x and ('p-plan__slide__name' in x or 'title' in x or 'plan-name' in x))
                if name_tag:
                    nombre_plan = name_tag.get_text(strip=True)
                else:
                    nombre_plan = ' '.join(head_tag.get_text(separator=' ', strip=True).split())

                nombre_plan = re.sub(r'Plan Postpago\s*', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'S/\s*\d+\.\d+', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'al mes', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'x \d+ meses', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'Precio regular:.*?(Ahorra \d+%)?', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'Bono \d+ GB', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'Exclusivo online', '', nombre_plan, flags=re.IGNORECASE)
                nombre_plan = re.sub(r'\*', '', nombre_plan).strip()
                # --- NUEVA LÍNEA PARA ELIMINAR "Ahorra" ---
                nombre_plan = re.sub(r'Ahorra\s*\d*%', '', nombre_plan, flags=re.IGNORECASE).strip()
                nombre_plan = re.sub(r'Ahorra', '', nombre_plan, flags=re.IGNORECASE).strip()
                # --- FIN NUEVA LÍNEA ---
                nombre_plan = re.sub(r'\s{2,}', ' ', nombre_plan).strip()

                if not nombre_plan:
                    nombre_plan = f"Plan Postpago {precio}" if precio != 'N/A' else 'Plan Postpago Desconocido'


            # 3. Extraer Gigas / Datos
            gigas_cantidad_tag = plan_element.find('span', class_='p-plan__slide__cantidad')
            if gigas_cantidad_tag:
                extracted_gigas = gigas_cantidad_tag.get_text(strip=True).replace('\n', ' ').strip()
                if "GB" in extracted_gigas.upper() or re.match(r'^\d+(\.\d+)?\s*GB$', extracted_gigas, re.IGNORECASE):
                    gigas = extracted_gigas
                elif "Bono" in extracted_gigas and "GB" in extracted_gigas:
                    gigas = extracted_gigas

            if gigas == 'N/A':
                ilimitado_tag = plan_element.find(['span', 'div', 'h3', 'p'], class_=lambda x: x and ('p-plan__slide__gigas' in x or 'gigas-text' in x or 'data-info' in x))
                if ilimitado_tag and ("ilimitado" in ilimitado_tag.get_text().lower() or "sin límites" in ilimitado_tag.get_text().lower()):
                    gigas = "Ilimitados"
                else:
                    text_content_lower = plan_element.get_text(separator=' ', strip=True).lower()
                    if "ilimitado" in text_content_lower and ("datos" in text_content_lower or "gigas" in text_content_lower):
                        gigas = "Ilimitados"
                    else:
                        match_gb = re.search(r'(\d+)\s*gb', text_content_lower)
                        if match_gb:
                            gigas = f"{match_gb.group(1)} GB"
                        else:
                            match_bono_gb = re.search(r'(bono\s*\d+\s*gb\s*x\s*\d+\s*meses)', text_content_lower)
                            if match_bono_gb:
                                gigas = match_bono_gb.group(1).replace('x', 'x ')

            # 4. Extraer Apps Ilimitadas
            apps_ttl_tag = plan_element.find('p', class_='p-plan__slide__apps__ttl')
            if apps_ttl_tag:
                apps_ilimitadas_text = apps_ttl_tag.get_text(strip=True)
                apps_ilimitadas_text = re.sub(r'\s*\n\s*', ' ', apps_ilimitadas_text).strip()
            else:
                apps_ilimitadas_list_temp = []
                apps_container = plan_element.find('div', class_='p-plan__slide__apps')
                if apps_container:
                    app_elements = apps_container.find_all(['img', 'span', 'i', 'p'], class_=lambda x: x and ('app-icon' in x or 'unlimited-app-icon' in x or 'logo-app' in x or 'app-name' in x))
                    for app_el in app_elements:
                        app_name = app_el.get('alt') or app_el.get('title') or app_el.get_text(strip=True)
                        if app_name and app_name.strip():
                            apps_ilimitadas_list_temp.append(re.sub(r'\s*\n\s*', ' ', app_name).strip())

                if not apps_ilimitadas_list_temp:
                    text_content_full = plan_element.get_text(separator=' ', strip=True)
                    match_apps_text = re.search(r'(?:Apps|Redes Sociales)\s+Ilimitadas(?::\s*(.*?))?(?=[.\n]|$)', text_content_full, re.IGNORECASE | re.DOTALL)
                    if match_apps_text:
                        if match_apps_text.group(1):
                            apps_ilimitadas_list_temp.extend([re.sub(r'\s*\n\s*', ' ', app.strip()).strip() for app in match_apps_text.group(1).split(',') if app.strip()])
                        else:
                            apps_ilimitadas_list_temp.append(re.sub(r'\s*\n\s*', ' ', match_apps_text.group(0).replace(":", "").strip()).strip())

                    keywords = ["WhatsApp", "Facebook", "Instagram", "TikTok", "Spotify", "Netflix", "Youtube", "Waze", "Twitter"]
                    for keyword in keywords:
                        if f"{keyword} Ilimitado" in text_content_full or f"Acceso ilimitado a {keyword}" in text_content_full:
                            if keyword.lower() not in [a.lower() for a in apps_ilimitadas_list_temp]:
                                apps_ilimitadas_list_temp.append(keyword)

                if apps_ilimitadas_list_temp:
                    apps_ilimitadas_text = ", ".join(sorted(list(set(apps_ilimitadas_list_temp))))

            # 5. Extraer Otros Beneficios (Minutos/Llamadas, SMS y otros) - LÓGICA MEJORADA Y MÁS GRANULAR
            all_benefit_texts_raw = []

            benefit_text_tags_specific = plan_element.find_all('p', class_='stefa-parrilla_blanco--body-texto')
            for tag in benefit_text_tags_specific:
                all_benefit_texts_raw.append(tag.get_text(strip=True))

            details_container = plan_element.find('div', class_='p-plan__slide__details')
            if details_container:
                general_benefit_tags = details_container.find_all(['li', 'p', 'span', 'div'], class_=lambda x: x and ('benefit-item' in x or 'feature-row' in x or 'text-benefit' in x or 'item-detail' in x or 'body-text' in x or 'plan-detail' in x))
                for tag in general_benefit_tags:
                    all_benefit_texts_raw.append(tag.get_text(strip=True))

            full_plan_text = plan_element.get_text(separator=' ', strip=True)
            all_benefit_texts_raw.append(full_plan_text)

            processed_texts = set()

            for raw_text in all_benefit_texts_raw:
                cleaned_text = re.sub(r'\s*\n\s*', ' ', raw_text).strip()
                if not cleaned_text or cleaned_text in processed_texts:
                    continue
                processed_texts.add(cleaned_text)

                text_lower = cleaned_text.lower()

                # --- Extracción granular de Minutos/Llamadas ---
                if llamadas_encontradas is None:
                    match_calls = re.search(r'(llamadas ilimitadas Perú(?:,)?(?: \d+ minutos para Usa y Canadá)?|minutos ilimitados Perú(?:,)?(?: \d+ para Usa y Canadá)?|llamadas ilimitadas a todo destino nacional|minutos ilimitados a todo destino nacional)', text_lower, re.IGNORECASE)
                    if match_calls:
                        llamadas_encontradas = match_calls.group(0).replace('perú,', 'Perú,').replace('usa y canadá', 'Usa y Canadá').replace('minutos para', 'minutos para ').strip()
                    elif 'llamadas ilimitadas' in text_lower:
                         llamadas_encontradas = 'Llamadas ilimitadas'
                    elif re.search(r'(\d+)\s*minutos\s*para\s*(usa|canadá|internacionales)', text_lower, re.IGNORECASE):
                        llamadas_encontradas = "Minutos internacionales (especificar cantidad)"

                # --- Extracción granular de SMS ---
                if sms_encontrados is None:
                    match_sms = re.search(r'(\d+)\s*sms|(sms ilimitados)', text_lower, re.IGNORECASE)
                    if match_sms:
                        if match_sms.group(1):
                            sms_encontrados = f"{match_sms.group(1)} SMS"
                        else:
                            sms_encontrados = "SMS ilimitados"

                if len(cleaned_text) > 10 and \
                   not (llamadas_encontradas and llamadas_encontradas in cleaned_text) and \
                   not (sms_encontrados and sms_encontrados in cleaned_text) and \
                   "gb" not in text_lower and "gigas" not in text_lower and \
                   "apps" not in text_lower and "precio" not in text_lower and \
                   "plan" not in text_lower and "bono" not in text_lower:

                    is_duplicate_or_classified = False
                    for existing_benefit_key, existing_benefit_value in otros_beneficios.items():
                        if cleaned_text == existing_benefit_value or cleaned_text in existing_benefit_value or existing_benefit_value in cleaned_text:
                            is_duplicate_or_classified = True
                            break

                    if not is_duplicate_or_classified:
                        if "apps ilimitadas" not in text_lower and "internet + llamadasilimitadas" not in text_lower and \
                           "bono" not in text_lower and "vigencia" not in text_lower and \
                           "roaming" not in text_lower and "streaming" not in text_lower:

                            key_found = False
                            for k, v in otros_beneficios.items():
                                if k.startswith('Otro Beneficio') and (cleaned_text in v or v in cleaned_text):
                                    key_found = True
                                    break
                            if not key_found:
                                otros_beneficios[f'Otro Beneficio {len([k for k in otros_beneficios if k.startswith("Otro Beneficio")]) + 1}'] = cleaned_text


            # Post-procesamiento para apps_ilimitadas_text que contiene información de llamadas
            if "llamadasilimitadas" in apps_ilimitadas_text.lower() and llamadas_encontradas is None:
                llamadas_encontradas = "Llamadas ilimitadas"
                apps_ilimitadas_text = re.sub(r'Internet \+ llamadasilimitadas', 'Internet', apps_ilimitadas_text, flags=re.IGNORECASE).strip()
                if apps_ilimitadas_text == 'Internet' or not apps_ilimitadas_text:
                    apps_ilimitadas_text = 'N/A'

            # Asignar los valores finales a otros_beneficios
            if llamadas_encontradas:
                otros_beneficios['Minutos/Llamadas'] = llamadas_encontradas
            if sms_encontrados:
                otros_beneficios['SMS'] = sms_encontrados


            # --- AGREGAR LOS DATOS DEL PLAN A LA LISTA ---
            planes_data.append({
                'Nombre del Plan': nombre_plan,
                'Precio (S/)': precio,
                'Gigas': gigas,
                'Apps Ilimitadas': apps_ilimitadas_text,
                **otros_beneficios
            })
            print(f"Extraído: Plan='{nombre_plan}', Precio='{precio}', Gigas='{gigas}'")

    except Exception as e:
        print(f"Ocurrió un error en la ejecución de Selenium: {e}")
        print("Revisa los selectores CSS y la conexión a internet de Colab.")
    finally:
        if driver:
            driver.quit()

    return planes_data

if __name__ == "__main__":
    print("Iniciando extracción de planes Postpago Movistar en Google Colab...")
    planes = extraer_planes_movistar_colab()
    if planes:
        print("\n--- Planes extraídos ---")
        for plan in planes:
            print(plan)

        df = pd.DataFrame(planes)
        df.to_csv("planes_movistar_postpago.csv", index=False)
        print("\nDatos guardados en planes_movistar_postpago.csv")
        print("Puedes descargarlo haciendo clic en el icono de 'Archivos' (carpeta) a la izquierda en Colab.")
    else:
        print("No se pudieron extraer los planes. Revisa los mensajes de error y los selectores finales.")

Iniciando extracción de planes Postpago Movistar en Google Colab...
Navegador abierto en Colab y cargando la página: https://www.movistar.com.pe/movil/postpago/planes-postpago
Elemento de precio detectado. Esperando que carguen más elementos...
Página cargada y elementos principales detectados. Extrayendo HTML...
Se encontraron 18 posibles contenedores de planes.
Extraído: Plan='Plan Postpago S/ 69.90', Precio='S/ 69.90', Gigas='Ilimitados'
Extraído: Plan='Plan Postpago S/ 39.90', Precio='S/ 39.90', Gigas='Ilimitados'
Extraído: Plan='Plan Postpago S/ 49.90', Precio='S/ 49.90', Gigas='Ilimitados'
Extraído: Plan='Plan Postpago S/ 59.90', Precio='S/ 59.90', Gigas='Ilimitados'
Extraído: Plan='Plan Postpago S/ 79.90', Precio='S/ 79.90', Gigas='Ilimitados'
Extraído: Plan='Plan Postpago S/ 99.90', Precio='S/ 99.90', Gigas='Ilimitados'
Extraído: Plan='Plan Postpago S/ 49.95', Precio='S/ 49.95', Gigas='135 GB'
Extraído: Plan='Plan Postpago S/ 39.95', Precio='S/ 39.95', Gigas='120 GB'
Extraído: 